In [1]:
import pymysql.cursors
import pandas as pd

In [2]:
conn = pymysql.connect(host='localhost', user='root', 
                       password='1111', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

    with conn.cursor() as curs:
      sql = "SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)
      #df.to_csv('query.csv')

finally:

   conn.close()

     total_bill   tip     sex smoker  day    time  size
0         16.99  1.01  Female     No  Sun  Dinner     2
1         10.34  1.66    Male     No  Sun  Dinner     3
2         21.01  3.50    Male     No  Sun  Dinner     3
3         23.68  3.31    Male     No  Sun  Dinner     2
4         24.59  3.61  Female     No  Sun  Dinner     4
..          ...   ...     ...    ...  ...     ...   ...
247       20.65  3.35    Male     No       Dinner     5
248       20.65  3.35    Male     No       Dinner     4
249       20.65  3.35    Male     No         None     4
250       20.65  3.35    Male     No         None     5
251       20.65  3.35    Male     No       Dinner     4

[252 rows x 7 columns]


In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          2
size          0
dtype: int64

In [4]:
#encoding  #inplace를 사용하면 원본에 적용이 된다. 데이터를 copy하여 변환하는 것이 아니라 원본 데이터에 직접 변환작업을 진행하고 메모리에 적용이 됨

In [5]:
df['sex'].replace({'Female':0, 'Male':1}, inplace=True)
df["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)
df["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [6]:
import numpy as np

In [7]:
df.replace('', np.nan, inplace=True)

In [8]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size
count,252.000000,252.000000,252.000000,252.000000,246.000000,250.000000,252.000000
mean,19.757937,2.993452,0.650794,0.369048,1.735772,0.728000,2.611111
std,8.782405,1.370859,0.477668,0.483507,1.156746,0.445883,0.977616
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,13.385000,2.000000,0.000000,0.000000,0.250000,0.000000,2.000000
50%,17.855000,3.000000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,24.022500,3.502500,1.000000,1.000000,3.000000,1.000000,3.000000
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000


In [9]:
from sklearn.impute import SimpleImputer
data = df.values

In [10]:
y = df['tip']  #Target variable -> Tip

In [11]:
X = df.drop('tip', axis = 1)
X.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [12]:
y.describe()

count    252.000000
mean       2.993452
std        1.370859
min        1.000000
25%        2.000000
50%        3.000000
75%        3.502500
max       10.000000
Name: tip, dtype: float64

In [13]:
data=X.values # Train data

In [14]:
y=y.values

In [15]:
sum(np.isnan(data).flatten())

8

In [16]:
sum(np.isnan(y).flatten())

0

In [17]:
imputer = SimpleImputer(strategy='median')

In [18]:
imputer.fit(data)

SimpleImputer(strategy='median')

In [19]:
data_trans = imputer.transform(data) # Imputed data set

In [20]:
#RFE

In [21]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [22]:
estimator = SVR(kernel="linear")

In [23]:
rfe = RFE(estimator, n_features_to_select=4)

In [24]:
selector=rfe.fit(data_trans, y)

In [25]:
selector.support_

array([ True,  True, False, False,  True,  True])

In [26]:
for i in range(data_trans.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

Column: 0,  Rank: 1
Column: 1,  Rank: 1
Column: 2,  Rank: 3
Column: 3,  Rank: 2
Column: 4,  Rank: 1
Column: 5,  Rank: 1


In [27]:
df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size'])

In [28]:
df1.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [29]:
df1.drop(['smoker', 'day'], axis=1)

,total_bill,sex,time,size
0,16.99,0.0,1.0,2.0
1,10.34,1.0,1.0,3.0
2,21.01,1.0,1.0,3.0
3,23.68,1.0,1.0,2.0
4,24.59,0.0,1.0,4.0
...,...,...,...,...
247,20.65,1.0,1.0,5.0
248,20.65,1.0,1.0,4.0
249,20.65,1.0,1.0,4.0
250,20.65,1.0,1.0,5.0


In [31]:
#PCA

In [32]:
from sklearn.decomposition import PCA

In [33]:
data_trans.shape

(252, 6)

In [34]:
trans = PCA(n_components=4)
X_dim = trans.fit_transform(data_trans)
print(X_dim[:3, :])

[[-2.77710332 -1.23648462 -0.48316348  0.71891134]
 [-9.33894108 -1.63677018  0.88446339 -0.2603256 ]
 [ 1.30542375 -1.30900723  0.25706786 -0.22802118]]


In [35]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [36]:
fs = SelectKBest(score_func=f_regression, k=4)

In [37]:
X_selected = fs.fit_transform(data_trans, y)

In [38]:
parameters = fs.get_params(deep=True)

In [39]:
parameters

{'k': 4,
 'score_func': <function sklearn.feature_selection._univariate_selection.f_regression(X, y, *, center=True)>}

In [40]:
parameters['score_func']

<function sklearn.feature_selection._univariate_selection.f_regression(X, y, *, center=True)>

In [41]:
print(X_selected.shape)

(252, 4)


In [42]:
X_selected.shape[1]

4

In [43]:
df_rfe_selected = pd.DataFrame(X_selected, columns=['total_bill', 'day', 'time', 'size'])

In [44]:
df_rfe_selected.describe()

,total_bill,day,time,size
count,252.000000,252.000000,252.000000,252.000000
mean,19.757937,1.742063,0.730159,2.611111
std,8.782405,1.143550,0.444761,0.977616
min,3.070000,0.000000,0.000000,1.000000
25%,13.385000,1.000000,0.000000,2.000000
50%,17.855000,2.000000,1.000000,2.000000
75%,24.022500,3.000000,1.000000,3.000000
max,50.810000,3.000000,1.000000,6.000000
